In [19]:
import pandas as pd

df = pd.read_csv('./data/ikine((0.05, 0.25), (-0.25, 0.25), 0.12, 10).csv')
df.head()

,x,y,z,t0,t1,t2,t3,x0,y0,x1,...,x2,y2,x3,y3,xc,yc,diagonal,b64_image,width,height
0,0.05,-0.24,0.12,1.768289,37.711826,81.025619,43.313793,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640,480
1,0.06,-0.24,0.12,4.036243,35.196067,77.045020,41.848953,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640,480
2,0.07,-0.24,0.12,6.260205,31.753588,71.469230,39.715641,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640,480
3,0.10,-0.23,0.12,13.498566,30.528690,69.449178,38.920487,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640,480
4,0.12,-0.22,0.12,18.610460,30.845048,69.972724,39.127676,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640,480


In [21]:
df = df.drop(columns=['b64_image'])
df

,x,y,z,t0,t1,t2,t3,x0,y0,x1,y1,x2,y2,x3,y3,xc,yc,diagonal,width,height
0,0.05,-0.24,0.12,1.768289,37.711826,81.025619,43.313793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640,480
1,0.06,-0.24,0.12,4.036243,35.196067,77.045020,41.848953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640,480
2,0.07,-0.24,0.12,6.260205,31.753588,71.469230,39.715641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640,480
3,0.10,-0.23,0.12,13.498566,30.528690,69.449178,38.920487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640,480
4,0.12,-0.22,0.12,18.610460,30.845048,69.972724,39.127676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640,480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,0.06,0.04,0.12,113.690068,145.273369,172.153766,26.880397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640,480
770,0.05,0.04,0.12,118.659808,149.754587,172.550744,22.796157,262.0,365.0,230.0,336.0,271.0,292.0,303.0,318.0,266.0,328.0,73.552702,640,480
771,0.05,0.05,0.12,125.000000,146.064346,172.247991,26.183645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640,480
772,0.05,0.03,0.12,110.963757,152.791769,172.616067,19.824298,263.0,360.0,235.0,329.0,280.0,290.0,309.0,320.0,271.0,325.0,72.034714,640,480


In [22]:
from sklearn.model_selection import train_test_split

df = df.dropna()

y = df[['t0', 't1', 't2', 't3']]

X = df.drop(columns=['t0', 't1', 't2', 't3'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neural_network import MLPRegressor

# Parâmetros do RandomizedSearchCV
param_distributions = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100, 100), (50, 50, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [500, 1000],
    'alpha': [0.0001, 0.001, 0.01]
}

mlp = MLPRegressor(random_state=42)

random_search = RandomizedSearchCV(
    mlp, param_distributions, 
    n_iter=10, scoring='neg_mean_squared_error', 
    cv=3, verbose=2, random_state=42,
    return_train_score=True, n_jobs=-1
)

random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_

y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Erro quadrático médio (MSE):", mse)
print("Erro absoluto médio (MAE):", mae)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


c:\Users\ruben\Desktop\2024_2\auto\.auto\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
3 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\ruben\Desktop\2024_2\auto\.auto\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\ruben\Desktop\2024_2\auto\.auto\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ruben\Desktop\2024_2\auto\.auto\Lib\site-packages\sklear

Erro quadrático médio (MSE): 9.33841621783322
Erro absoluto médio (MAE): 1.9142119212675235
